In [12]:
"""imports"""
import os
from ansys.edb.database import Database
from ansys.edb.session import launch_session
import ansys.edb as edb
import psutil

In [13]:
"""configs"""
ROOT = os.getcwd()
EXE_ROOT = os.environ['ANSYSEM_ROOT232']
ROOT = os.getcwd()
EDB_FILE = os.path.join(ROOT,'data','meshed_ground.aedb')
out = os.path.join(ROOT,'data','unite_meshed_ground.aedb')

In [14]:
"""launch session"""
PROCNAME = "EDB_RPC_Server.exe"
try:
    for proc in psutil.process_iter():
        if proc.name() == PROCNAME:
            proc.kill()
except:
    pass
session = launch_session(EXE_ROOT, 50051)

In [15]:
"""open db"""
db = Database.open(EDB_FILE,False)
cell = db.circuit_cells[0]
layout = cell.layout
layerStats = []
lc = layout.layer_collection
layers = lc.get_layers()
layer_names = [i.name for i in layers]
nets = layout.nets
pgnets = ['GND']

In [16]:
"""Merge polygons in PG nets and save db"""
for net in nets:
        netName = net.name
        if netName in pgnets:
            prims = list(net.primitives)
            layers = list(set([i.layer.name for i in prims]))
            Void = {};Poly = {};Hole={}
            for layer in layers:
                Poly[layer] = []
                Hole[layer] = []
                for prim in prims:
                    if prim.layer.name==layer:
                        Poly[layer].append(prim.polygon_data)
                        if prim.has_voids:
                            for v_prim in prim.voids:
                                Hole[layer].append(v_prim.get_polygon_data())
                united_poly = edb.geometry.polygon_data.PolygonData.unite(Poly[layer])[0]
                for i in Hole[layer]:
                    united_poly = united_poly.subtract(united_poly,i)[0]
                tmpPoly = edb.primitive.primitive.Polygon.create(layout,layer,net,united_poly)
            for primitive in prims:
                primitive.delete()
db.save_as(out)

In [17]:
"""End the session"""
session.disconnect()